<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [0]:
from skimage.io import imread_collection, imread
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
mountains = imread_collection(
    './data/mountain/*.jpg').concatenate()

forests = imread_collection(
    './data/forest/*.jpg').concatenate()

In [0]:
mountains = resize(mountains, (374, 256, 256))

In [0]:
forests = resize(forests, (328, 256, 256))

In [0]:
images = np.concatenate([forests, mountains])

In [0]:
image_labels = np.zeros(328)
image_labels = np.concatenate([image_labels, np.ones(374)])

## Instatiate Model

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

In [0]:
for layer in resnet.layers:
    layer.trainable = False

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

## Fit Model

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_loss', min_delta=.1, patience=5, restore_best_weights=True)

In [26]:
out = model.fit(images, image_labels, batch_size=2,
                epochs=999, validation_split=.2, callbacks=[stop])

Epoch 1/999
281/281 [==============================] - 5s 17ms/step - loss: 0.2068 - accuracy: 0.9109 - val_loss: 0.3708 - val_accuracy: 0.8723
Epoch 2/999
281/281 [==============================] - 5s 16ms/step - loss: 0.1858 - accuracy: 0.9251 - val_loss: 0.2419 - val_accuracy: 0.9078
Epoch 3/999
281/281 [==============================] - 5s 16ms/step - loss: 0.1751 - accuracy: 0.9358 - val_loss: 0.6440 - val_accuracy: 0.7234
Epoch 4/999
281/281 [==============================] - 4s 16ms/step - loss: 0.1796 - accuracy: 0.9323 - val_loss: 0.7052 - val_accuracy: 0.7163
Epoch 5/999
281/281 [==============================] - 4s 16ms/step - loss: 0.1543 - accuracy: 0.9376 - val_loss: 0.5344 - val_accuracy: 0.7943
Epoch 6/999
281/281 [==============================] - 4s 16ms/step - loss: 0.1762 - accuracy: 0.9269 - val_loss: 1.8735 - val_accuracy: 0.2979
Epoch 7/999
281/281 [==============================] - 5s 16ms/step - loss: 0.2245 - accuracy: 0.9144 - val_loss: 0.5571 - val_accuracy:

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, BatchNormalization

In [0]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [13]:
images[0].shape

(256, 256, 3)

In [27]:
model = Sequential()

model.add(Conv2D(128, (3,3), input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))
model.add(Flatten())

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 254, 128)     3584      
_________________________________________________________________
batch_normalization_4 (Batch (None, 254, 254, 128)     512       
_________________________________________________________________
activation_5 (Activation)    (None, 254, 254, 128)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 254, 254, 128)     0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 127, 127, 128)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 125, 64)      73792     
_________________________________________________________________
batch_normalization_5 (Batch (None, 125, 125, 64)     

In [0]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [0]:
stop2 = EarlyStopping(monitor='val_loss', min_delta=.1, patience=5, restore_best_weights=True)

In [34]:
history = model.fit(images, image_labels, epochs=999,
                    batch_size=16, validation_split=.2, callbacks=[stop2])

Epoch 1/999
36/36 [==============================] - 3s 88ms/step - loss: 0.2160 - accuracy: 0.9180 - val_loss: 0.2840 - val_accuracy: 0.9787
Epoch 2/999
36/36 [==============================] - 3s 83ms/step - loss: 0.1635 - accuracy: 0.9394 - val_loss: 0.1701 - val_accuracy: 1.0000
Epoch 3/999
36/36 [==============================] - 3s 82ms/step - loss: 0.2502 - accuracy: 0.9376 - val_loss: 0.1242 - val_accuracy: 1.0000
Epoch 4/999
36/36 [==============================] - 3s 83ms/step - loss: 0.2526 - accuracy: 0.9162 - val_loss: 0.1035 - val_accuracy: 1.0000
Epoch 5/999
36/36 [==============================] - 3s 83ms/step - loss: 0.1667 - accuracy: 0.9447 - val_loss: 0.1404 - val_accuracy: 1.0000
Epoch 6/999
36/36 [==============================] - 3s 82ms/step - loss: 0.1899 - accuracy: 0.9340 - val_loss: 0.2085 - val_accuracy: 0.9858
Epoch 7/999
36/36 [==============================] - 3s 83ms/step - loss: 0.1075 - accuracy: 0.9590 - val_loss: 0.1522 - val_accuracy: 1.0000


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language